In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from netam.common import pick_device, print_parameter_count

In [2]:
# shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
shmoof_data_path = "/Users/matsen/data/shmoof_pcp_2023-11-30_MASKED.csv"
train_df, val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname="51") # , sample_count=5000)

In [3]:
kmer_length = 3
site_count = 500

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, site_count=site_count)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, site_count=site_count)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 2722 training examples and 2278 validation examples


In [4]:
# [I 2023-12-03 13:38:46,435] Trial 37 finished with value: 0.0603752824113363 and parameters: {'kernel_size': 19, 'dropout': 0.0, 'embedding_dim': 12, 'filter_count': 12}. Best is trial 37 with value: 0.0603752824113363.

cat_params = {
    "l2_regularization_coeff": [1e-6, 1e-5, 1e-4],
}
int_params = {
}
float_params = {
}
log_float_params = {
}
# Note that if anything appears below and above, the above gets priority.
fixed_hyperparams = {
    "kmer_length": kmer_length,
    "embedding_dim": 14,
    "filter_count": 25,
    "kernel_size": 15,
    "dropout_prob": 0.1,
    "batch_size": 1024,
    "learning_rate": 0.1,
    "min_learning_rate": 1e-3, # early stopping!
    "l2_regularization_coeff": 1e-6,
    "min_parameter_count": 1000,
    "max_parameter_count": 10000,
}
epochs = 30
optuna_steps = 50

hyper_burrito = framework.HyperBurrito(pick_device(), train_dataset, val_dataset, models.CNNModel, framework.SHMBurrito, epochs=epochs)

hyper_burrito.optuna_optimize(optuna_steps, cat_params, int_params, float_params, log_float_params, fixed_hyperparams)

[I 2023-12-15 04:39:11,310] A new study created in memory with name: no-name-54901fe1-f141-4c7e-88b8-c510cb5e7f92


Using Metal Performance Shaders


Epoch: 100%|██████████| 30/30 [00:28<00:00,  1.05it/s, loss_diff=-2.325e-05, lr=0.1, val_loss=0.07097]
[I 2023-12-15 04:39:41,357] Trial 0 finished with value: 0.06806854113504696 and parameters: {'l2_regularization_coeff': 1e-06}. Best is trial 0 with value: 0.06806854113504696.
Epoch:  87%|████████▋ | 26/30 [00:25<00:03,  1.04it/s, loss_diff=-6.058e-07, lr=3.2e-5, val_loss=0.07481] 
[I 2023-12-15 04:40:07,137] Trial 1 finished with value: 0.06712726947113126 and parameters: {'l2_regularization_coeff': 0.0001}. Best is trial 1 with value: 0.06712726947113126.
Epoch:  70%|███████   | 21/30 [00:20<00:08,  1.02it/s, loss_diff=-5.132e-06, lr=0.004, val_loss=0.07044]
[W 2023-12-15 04:40:28,296] Trial 2 failed with parameters: {'l2_regularization_coeff': 1e-06} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = 

KeyboardInterrupt: 